# **Solving the time-independent Schrödinger equation in 1D**

## Importation des libraries utilisé

In [218]:
import numpy as np

from math import pi, sqrt, sin, factorial
from scipy.integrate import quad

In [219]:
# Debuging mode ?
debug = False

In [220]:
# Typing
type Vector = np.ndarray[float]
type Family = list[Vector]
type Matrix = list[list[float]]

## Procédure de diagonalisation

### Procédure d'orthonormalisation utile pour la méthode de Davidson
On utlise l'algorithme de gramschmidt (https://fr.wikipedia.org/wiki/Algorithme_de_Gram-Schmidt)

In [221]:
def matrix_to_family(M: Matrix) -> Family:
    """ Transforme une matrice en une famille de vecteur constituer des colones de cette matrice """
    return np.array(M).T

def family_to_matrix(L: Family) -> Matrix:
    """ Transforme une famille de vecteur en matrice """
    return np.array(L).T

if debug:
    M = np.random.randint(100, size=(10,10))
    print("On veut transformé un matrice M : \n\n", M, "\n\nen famille de vecteur : \n")
    print(matrix_to_familly(M))
    print("\nOn l'a retransforme en matrice :\n")
    print(family_to_matrix(matrix_to_family(M)))

In [222]:
def ps(a: Vector, b: Vector) -> float:
    """
    Fait le produit scalaire canonique de ℝ^n expliquer dans
    https://fr.wikipedia.org/wiki/Produit_scalaire_canonique
    """
    sum = 0
    for i in range(len(a)):
        sum += a[i]*b[i]
    return sum

if debug:
    u = [1,2,3]
    v = [2,3,4]
    print(f"On fait le produit scalaire canonique de {u} avec {v} qui doit donner 20")
    print("résultat = ", ps(u, v))

In [223]:
def norm(u: Vector) -> float:
    """ Norme associé au produit scalaire canonique de ℝ^n """
    return ps(u, u)**(1/2)

if debug:
    u = np.array([1, 1, 1])
    print(f"On calcule la norme de {u} associé au produit saclaire canonique de ℝ^n qui doit donner √3")
    print("résultat = ", norm(u))

In [224]:
def orthonorm_vec(F: Family, v: Vector) -> Vector:
    """ Orthonomalise un vecteur par rapport à une famille de vecteurs orthonormaux """
    u = np.array(v)
    for vk in np.array(F):
        u = u - ps(vk, v)*vk
    return u/norm(u)

if debug:
    F = [[1, 0, 0],[0, 1, 0]]
    v = [1, 1, 1]
    print(f"Orthonormalise le vecteur {v} par rapport à la famille orthonormale \n{F}\n doit renvoyer le vecteur (0,0,1)")
    print("résultat = ", orthonorm_vec(F, v))    

In [225]:
def gram_schmidt(F: Family) -> Family:
    """ Aplique l'algorithme de gramschmidt pour orthonomaliser la famille quelconque F """
    ONF = []
    for e in F:
        ONF.append(orthonorm_vec(ONF, e))
    return np.array(ONF)

if debug:
    F = [[1, 0, 0],[1, 1, 0],[1, 1, 1]]
    print("On orthonormalise la famille F : ")
    print(F)
    print("On doit trouver [[1, 0, 0],[0, 1, 0],[0, 0, 1]]")
    print("résultat : ", gram_schmidt(F))

In [226]:
def gram_schmidt_matrix(M: Matrix) -> Matrix:
    """ Aplique l'algorithme de gramschmidt pour orthonomaliser la matrice M. """
    return family_to_matrix(gram_schmidt(matrix_to_family(M)))

if debug:
    M = np.random.randint(100,size=(3, 3))
    print("On veut orthonormaliser M :")
    print(M)
    print("résultat :")
    ONM = gram_schmidt_matrix(M)
    print(ONM)
    print("La matrice est orthonormale ?")
    print("norme : ", norm(ONM[:,0].T))
    print("norme : ", norm(ONM[:,1].T))
    print("norme : ", norm(ONM[:,2].T))
    print("produit scalaire : ", ps(ONM[:,0].T, ONM[:,2].T,))
    print("produit scalaire : ", ps(ONM[:,0].T, ONM[:,1].T,))
    print("produit scalaire : ", ps(ONM[:,2].T, ONM[:,1].T,))

### Procédure de tri pour la méthode de Davidson
On utilise la méthode de trifusion qui est en complexité n*log(n) (https://fr.wikipedia.org/wiki/Tri_fusion)

In [227]:
def fusion(A: list, B: list) -> list:
    """ Procédure qui fusione deux listes trié """
    if len(A) == 0:
        return B
    elif len(B) == 0:
        return A
    elif A[0] <= B[0]:
        return [A[0]] + fusion(A[1:], B)
    else:
        return [B[0]] + fusion(A, B[1:])
if debug:
    A = [1, 3, 4, 7]
    B = [2, 3, 5]
    print(f"On fusione les listes {A} et {B} pour obtenir cette liste : [1, 2, 3, 3, 4, 5, 7]")
    print("résultat : ", fusion(A, B))

In [228]:
def triFusion(L):
    """ On tri une liste en utilisant le principe de diviser pour reigner """
    if len(L) == 1:
        return L
    else:
        return fusion(triFusion(L[:len(L)//2]) , triFusion(L[len(L)//2:]))

if debug:
    L = list(zip([1, 76, 71, 6, 25, 50, 20, 18, 84, 11],[1,1,1,1,1,1,1,1,1,1]))
    print("On tri la liste :", L, "selon les premières valeur du couple")
    print("résultat :", triFusion(L))

### Méthode de Davidson
Implémentation basé sur ces explications page 65 : https://www.irisa.fr/sage/bernard/publis/DAVIDSON94.pdf

In [229]:
def davidson(M: Matrix, m=3, l=1, seuil=1e-8, MIt=600) -> tuple:
    """
    M : la matrice à diagonaliser
    m : la précision de l'agorithme Km = {V0;MV0;M²V0;...}
    l : le nombre de vecteur/valeur propre à trouver
    seuil : précision de l'aproxiamtion des valeurs propres
    MIt : maximum d'itiération

    Fonction qui retourne le coupe ([valeur propre], [vecteur propre]) de
    la matrice M
    """
    np.set_printoptions(precision=3, suppress=True)

    # Initialisation
    M = np.array(M)
    N = len(M)
    V1 = np.eye(N, l)
    I = np.identity(N)

    if not(l<=m<=N):
        raise ValueError(f"value must be {l} <= {m} <= {N}")

    # Concstruction du sous espace réduit de (Kernal)
    v = [V1]

    # itération
    for k in range(MIt):
        # Résolution du sous-espace réduit
        T = np.dot(v[k].T, np.dot(M, v[k]))
        val, vec = np.linalg.eig(T)
        #val, vec = lanczos(T)
        Eigencouple = list(zip(val,vec))
        Eigencouple = triFusion(Eigencouple)[-l:] # Plus grande VP
        #Eigencouple = triFusion(Eigencouple)[:l] # Plus petite VP
        val, vec = list(zip(*Eigencouple))
        vec = np.array(vec).T
        val = np.array(val)

        x = np.array([[] for _ in range(N)])
        r = np.array([[] for _ in range(N)])
        t = np.array([[] for _ in range(N)])

        # Expansion du sous-espace
        for i in range(l):
            yi = vec[:, i].reshape((T.shape[0], 1))

            xi = np.dot(v[k], yi)
            x = np.concatenate((x, xi), axis=1)

            ri = val[i]*xi-np.dot(M, np.dot(v[k], yi))
            r = np.concatenate((r, ri), axis=1)

            # Seuil à 1e-300 pour eviter les problèmesde division par 0
            mu_i = [1/max(np.abs(val[i]-M[j, j]), 1e-200)
                    for j in range(N)]
            Ci = np.diag(mu_i)
            ti = np.dot(Ci, ri)
            t = np.concatenate((t, ti), axis=1)
                
        # Stop si l'algo converge        
        if np.linalg.norm(t) < seuil:
            print("Davidson converge")
            return val, x

        # Sinon on regénère une nouvelle matrice de passage
        if v[k].shape[0] <= m - l :
            v.append(gram_schmidt_matrix(np.concatenate((v[k], t), axis=1)).reshape(N, v[k].shape[1]+l))
        else:
            v.append(gram_schmidt_matrix(np.concatenate((x, t), axis=1)).reshape(N, 2*l))

    return val, x

if debug:
    M = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]
    print("On cherche les valeurs et vecteurs propres de : \n", M)
    print("résultat : ", davidson(M))
    print("résultat théorique :", np.linalg.eig(M))

### Méthode de Lanczos
Implémentation basé sur ces explications https://en.wikipedia.org/wiki/Lanczos_algorithm

In [255]:
# En construction
def lanczos():
    pass

## Construction des bases de résolution de l'équation

### Base des sinus
> ⚠️⚠️**Atention les calcules à la main sont sans doute faux car différant de la verssion numérique** ⚠️⚠️

Cette fonction permet d'initialiser la classe de B_sin en initialisant notament les valeurs des différentes variables.
Cette classe possède les méthodes suivante :

- init :
Cette fonction initialise les différentes variables associé à notre classe B_sin.

- ret_base :
Cette fonction renvoie la base composée de sinus de la taille désirée.

- prdt_scalaire_Ec :
Renvoie le produit de points calculé à la main de l'énergie cinétique de l'hamiltonien.

- prdt_scalaire_Ep :
Renvoie le produit point calculé à la main de l'énergie potentielle de l'hamiltonien.

In [231]:
class B_sin():
    """ Cette classe permet de calculer la base des sinus pour le calcul de l'hamiltonien """
    def __init__(self, N, L, V):
        """ Methode qui initialise la classe de l'objet B_sin() """
        self.N = N
        self.L = L
        self.V = V
        x = sqrt(2)
        self.k = eval(self.V)
    
    def ret_base(self):
        """ Methode qui calcule la liste des sinus qui formera la base """
        return [f'sqrt(2/{self.L}) * sin(({n}*pi*x)/{self.L})' for n in 
                range(self.N)]

    def prdt_scalaire_Ec(self, i, j):
        """ Methode qui donne le produit scalaire : <Φi|T̂|Φj> (calcul a la main) """
        if i == j:
            return - (j*pi)**2 / self.L
        else :
            return 0
    
    def prdt_scalaire_Ep(self, i, j):
        """ Methode qui donne le produit scalaire : <Φi|V̂|Φj> (calcul a la main) pour un potentiel harmonique"""
        if i == j :
            return (self.k * (self.L)**2) / 3
        else :
            return ((2 * self.k * (self.L)**2) / pi**2) * ( (((-1)**(i+j))/(i+j)**2) - (((-1)**(i-j))/(i-j)**2) )

if debug:
    N = 3 # La dimenssion de la base
    L = 1 # en U.A. est la taille du puit infini où les sinus sont vecteurs propres de l'hamiltonien
    m = 1 # en U.A. est la masse de la particule
    w = 1 # en U.A. est la pulsation de la vibration
    h = 1 # en U.A. est la constante de Plank
    k = m*(w**2) # la raideur du ressort
    V = f"({k}*x**2)/2" # le potentiel d'un puit infini
    B = B_sin(N, L, V)
    print(f"La base des sinus qui est base pour le puit infini de longueur {L} U.A. constituer de {N} vecteurs est :")
    print(B.ret_base())
    i = 0
    j = 0
    print(f"La valeur de l'énergie cinétique du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ec(i,j))
    print(f"La valeur de l'énergie potentiel du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ep(i,j))

### Other basis

We have also drafted a basis with a harmonic oscillator, which you will find commented on at the end of the Python document 'base'.  
**The different sources:**  
* https://fr.wikipedia.org/wiki/Oscillateur_harmonique_quantique  
* https://fr.wikipedia.org/wiki/Polyn%C3%B4me_d%27Hermite



### Base quelconque
On veut créer ici une classe base qui créer une base en compréhanssion 

Notre class base aurra besoin de calculer la dériver seconde d'une fonction en un point

In [232]:
def deriv(f : str, x : float, h = 1e-3):
    """ 
    Fonction qui calcule la dériver de la fonction f en x.
    f est un string.
    Exemple d'utilisation pour deriver la fonction sinus en 0:
    f = 'np.sin(x)'
    x = 0.
    resultat = deriv(f,x)
    """
    x += h
    res = eval(f)
    x -= 2 * h
    res -= eval(f)
    return res/(2*h)

def derivderiv(func : str, x : float, h = 1e-3):
    """
    Fonction qui calcule la dériver seconde de la fonction f en x.
    f est un string.
    Exemple d'utilisation pour deriver deux fois la fonction sinus en 0:
    f = 'np.sin(x)'
    x = 0.
    resultat = derivderiv(f,x)
    """
    return (deriv(func,x+h)-deriv(func,x-h))/(2*h)

if debug:
    f = "np.sin(x)"
    x = 0.
    print(f"La dériver de {f} en {x} est ", deriv(f,x))
    print(f"La dériver seconde de {f} en {x} est ", derivderiv(f,x))

In [233]:
class base():
    """ Cette classe permet de calculer une base quelconque pour le calcul de l'hamiltonien """
    def __init__(self, N : int, func : str, var : str, pot : str, L : float):
        """ 
        Methode qui initialise la classe de l'objet base.
        N : Le nombre de vecteur de la base
        func : est un string de la fonction comme si on l'écrivait. ex: func = "np.sin(n*x)"
        var : est la variable d'itération de la base. ex: avec l'exemple précédant on itère selon la variable n : (sin(x), sinc(2x), sin(3x),...)
        pot : le potentiel associé au problème défini selon les même restriction que func
        L : La largeur de la boîte de potentiel qui comprend le potentiel définit
        """
        self.V = pot
        # create the base
        self.base =[]
        self.L = L

        function = func.split(var)
        for n in range(1,N+1):
            f = ''
            for i in range(len(function)):
                if i != len(function) - 1:
                    f += function[i] + str(n)
                else:
                    f += function[i]
            self.base.append(f)

    def prdt_scalaire_Ec(self, i, j) -> float:
        """ Methode qui donne le produit scalaire : <Φi|T̂|Φj> """
        def func(x):
            x = x
            return eval(self.base[i]) * derivderiv(self.base[j],x)
        return quad(func, -self.L, self.L)[0]
    
    def prdt_scalaire_Ep(self, i, j) -> float:
        """ Methode qui donne le produit scalaire : <Φi|V̂|Φj>  pour un potentiel quelconque"""
        def func(x):
            x = x
            return eval(self.base[i]+'*'+self.V+'*'+self.base[j])
        return quad(func, -self.L, self.L)[0]

if debug:
    N = 3 # La dimenssion de la base
    L = 1 # en U.A. est la taille du puit infini où les sinus sont vecteurs propres de l'hamiltonien
    m = 1 # en U.A. est la masse de la particule
    w = 1 # en U.A. est la pulsation de la vibration
    h = 1 # en U.A. est la constante de Plank
    k = m*(w**2) # la raideur du ressort
    V = f"({k}*x**2)/2" # le potentiel d'un puit infini

    f = "np.sin(a*x)"
    var = "a"
    
    # Définition de la base
    B = base(N, f, var, V, L)
    print(f"La base des {f} qui est contenue dans un puit infini de longueur {L} U.A. constituer de {N} vecteurs est :")
    print(B.base)
    i = 0
    j = 0
    print(f"La valeur de l'énergie cinétique du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ec(i,j))
    print(f"La valeur de l'énergie potentiel du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ep(i,j))

## Routine de résolution
### Procédure qui détermine l'hamiltonien du système associé à la base de description

In [234]:
def CalculHamiltonian(base, N):

    H = np.zeros((N,N))

    for i in range(N):
        for j in range(i + 1):
            H[i,j] = base.prdt_scalaire_Ec(i, j) +  base.prdt_scalaire_Ep(i, j)

    H = H + H.T - np.diag(np.diag(H))

    return H

In [250]:
### Constantes et fonctions
N = 10 # La dimenssion de la base
L = 1 # en U.A. est la taille du puit infini où les sinus sont vecteurs propres de l'hamiltonien
m = 1 # en U.A. est la masse de la particule
w = 1 # en U.A. est la pulsation de la vibration
h = 1 # en U.A. est la constante de Plank
k = m*(w**2) # la raideur du ressort
V = f"({k}*x**2)/2" # le potentiel d'un puit infini

func = f'np.sqrt(2/{L})*np.sin( (a*np.pi*x) /{L})'
var = "a"

In [254]:
B = base(N, func, var, V, L)
print("La base du système est :")
print(B.base)

H = CalculHamiltonian(B, N//2)
print()
print("L'hamiltonien associé est :")
print(H)

print()
print("Résolution du système en cours ...")
E, phi = davidson(H)
val, vec = np.linalg.eig(H)

print()
print("Résolution terminer.")
print("Energie numerique : ", E)
print("Energie numerique numpy : ", val)
print("Energie analitique : ", h*w/2)

La base du système est :
['np.sqrt(2/1)*np.sin( (1*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (2*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (3*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (4*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (5*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (6*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (7*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (8*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (9*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (10*np.pi*x) /1)']

L'hamiltonien associé est :
[[ -19.456   -0.18     0.038   -0.014    0.007]
 [  -0.18   -78.635   -0.195    0.045   -0.018]
 [   0.038   -0.195 -177.32    -0.199    0.047]
 [  -0.014    0.045   -0.199 -315.481   -0.2  ]
 [   0.007   -0.018    0.047   -0.2   -493.108]]

Résolution du système en cours ...

Résolution terminer.
Energie numerique :  [-19.456]
Energie numerique numpy :  [ -19.456 -493.109  -78.635 -177.32  -315.481]
Energie analitique :  0.5
